In [5]:
import os
import sys
import socket
import subprocess
import platform
import psycopg2
from typing import Dict, Optional

def check_postgres_service() -> bool:
    """Check if PostgreSQL service is running."""
    system = platform.system().lower()
    try:
        if system == 'windows':
            output = subprocess.check_output('sc query postgresql', shell=True)
            return b'RUNNING' in output
        elif system in ['linux', 'darwin']:
            output = subprocess.check_output(['ps', 'aux'])
            return b'postgres' in output
    except subprocess.CalledProcessError:
        return False
    return False

def check_port_availability(port: int = 5432) -> bool:
    """Check if the specified port is open and accepting connections."""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.connect(('localhost', port))
        sock.close()
        return True
    except socket.error:
        return False
    finally:
        sock.close()

def get_env_vars() -> Dict[str, Optional[str]]:
    """Get all relevant PostgreSQL environment variables."""
    env_vars = {
        'PGHOST': os.getenv('PGHOST', 'localhost'),
        'PGDATABASE': os.getenv('PGDATABASE'),
        'PGUSER': os.getenv('PGUSER'),
        'PGPASSWORD': os.getenv('PGPASSWORD'),
        'PGPORT': os.getenv('PGPORT', '5432')
    }
    return env_vars

def test_connection(env_vars: Dict[str, Optional[str]]) -> None:
    """Test PostgreSQL connection with provided environment variables."""
    try:
        conn = psycopg2.connect(
            dbname=env_vars['PGDATABASE'],
            user=env_vars['PGUSER'],
            password=env_vars['PGPASSWORD'],
            host=env_vars['PGHOST'],
            port=env_vars['PGPORT']
        )
        conn.close()
        print("✓ Connection test successful")
    except psycopg2.Error as e:
        print("✗ Connection test failed:", str(e))

def main():
    """Main diagnostic function."""
    print("\n=== PostgreSQL Connection Diagnostic Report ===\n")
    
    # 1. Check environment variables
    print("1. Checking environment variables:")
    env_vars = get_env_vars()
    for var, value in env_vars.items():
        if var == 'PGPASSWORD' and value:
            print(f"  {var}: ****** (hidden)")
        else:
            print(f"  {var}: {value if value else 'Not set'}")
    
    # 2. Check if PostgreSQL service is running
    print("\n2. Checking PostgreSQL service:")
    if check_postgres_service():
        print("  ✓ PostgreSQL service is running")
    else:
        print("  ✗ PostgreSQL service is NOT running")
        print("  Suggestion: Start PostgreSQL service:")
        if platform.system().lower() == 'windows':
            print("    - net start postgresql")
        else:
            print("    - sudo service postgresql start")
    
    # 3. Check port availability
    port = int(env_vars['PGPORT']) if env_vars['PGPORT'] else 5432
    print(f"\n3. Checking port {port}:")
    if check_port_availability(port):
        print(f"  ✓ Port {port} is open")
    else:
        print(f"  ✗ Port {port} is not accessible")
        print("  Suggestions:")
        print("    - Verify PostgreSQL is running")
        print("    - Check if another service is using the port")
        print(f"    - Confirm PostgreSQL is configured to listen on port {port}")
    
    # 4. Test connection
    print("\n4. Testing database connection:")
    if all(env_vars.values()):
        test_connection(env_vars)
    else:
        print("  ✗ Missing required environment variables")
        print("  Suggestion: Set the following environment variables:")
        for var, value in env_vars.items():
            if not value:
                print(f"    - {var}")

if __name__ == "__main__":
    main()


=== PostgreSQL Connection Diagnostic Report ===

1. Checking environment variables:
  PGHOST: localhost
  PGDATABASE: Not set
  PGUSER: Not set
  PGPASSWORD: Not set
  PGPORT: 5432

2. Checking PostgreSQL service:
  ✗ PostgreSQL service is NOT running
  Suggestion: Start PostgreSQL service:
    - net start postgresql

3. Checking port 5432:
  ✓ Port 5432 is open

4. Testing database connection:
  ✗ Missing required environment variables
  Suggestion: Set the following environment variables:
    - PGDATABASE
    - PGUSER
    - PGPASSWORD
